In [100]:
# Load Libraries - Make sure to run this cell!
import pandas as pd
import numpy as np
import re, os
from string import printable
from sklearn import model_selection

#import gensim
import tensorflow as tf
from keras.models import Sequential, Model, model_from_json, load_model
from keras import regularizers
from keras.layers.core import Dense, Dropout, Activation, Lambda, Flatten
from keras.layers import merge
from keras.layers import Input, ELU, LSTM, Embedding, Convolution2D, MaxPooling2D, \
BatchNormalization, Convolution1D, MaxPooling1D, concatenate, merge
from keras.preprocessing import sequence
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras import initializers
from keras import backend as K
tf.python.control_flow_ops = tf
from pathlib import Path
import json

import warnings
warnings.filterwarnings("ignore")

In [101]:
from urllib.parse import urlparse

%matplotlib inline
cwd = os.getcwd()#To access the path of current working directory
path=cwd+"/"+"data"+"/"
data=pd.read_csv(path +"sample_10.csv")
data.head()
df=data
df.head()


,url,malicious
0,http://www.facebook-log1in.com/,1
1,http://getir.net/yg4t,1
2,http://crosscitydental.com/,0
3,http://www.tenttrails.com/jscripts/product.js,0
4,http://103.224.193.105/,0


### Preprocessing Raw URLs


In [102]:
# Initial Data Preparation URL

# Step 1: Convert raw URL string in list of lists where characters that are contained in "printable" are stored encoded as integer 
url_int_tokens = [[printable.index(x) + 1 for x in url if x in printable] for url in df.url]

# Step 2: Cut URL string at max_len or pad with zeros if shorter
max_len=75
X = sequence.pad_sequences(url_int_tokens, maxlen=max_len)
print(X)
# Step 3: Extract labels form df to numpy array
target = np.array(df.malicious)

print('Matrix dimensions of X: ', X.shape, 'Vector dimension of target: ', target.shape)

[[ 0  0  0 ..., 25 23 77]
 [ 0  0  0 ..., 17  5 30]
 [ 0  0  0 ..., 25 23 77]
 ..., 
 [ 0  0  0 ..., 25 23 77]
 [ 0  0  0 ..., 27 31 77]
 [ 0  0  0 ..., 25 30 83]]
Matrix dimensions of X:  (9998, 75) Vector dimension of target:  (9998,)


In [126]:
# GENERAL get layer dimensions for any model!
def print_layers_dims(model):
    l_layers = model.layers
    # Note None is ALWAYS batch_size
    for i in range(len(l_layers)):
        print(l_layers[i])
        print('Input Shape: ', l_layers[i].input_shape, 'Output Shape: ', l_layers[i].output_shape)


### Training and Evaluation


In [127]:
# Simple Cross-Validation: Split the data set into training and test data
X_train, X_test, target_train, target_test = model_selection.train_test_split(X, target, test_size=0.25, random_state=33)

#### Simple LSTM
Model features
1. LSTM 
2. Activation function :Sigmoid. 
3. Loss Function binary_crossentropy
4. number of Epochs 5

In [107]:
## Simple LSTM
def simple_lstm(max_len=75, emb_dim=32, max_vocab_len=100, lstm_output_size=32, W_reg=regularizers.l2(1e-4)):
    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')
    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len,
                dropout=0.2, W_regularizer=W_reg)(main_input) 

    # LSTM layer
    lstm = LSTM(lstm_output_size)(emb)
    lstm = Dropout(0.5)(lstm)
    
    # Output layer (last fully connected layer)
    output = Dense(1, activation='sigmoid', name='output')(lstm)

    # Compile model and define optimizer
    model = Model(input=[main_input], output=[output])
    
    sgd= SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
    
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    #model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [108]:
# Fit model and Cross-Validation, ARCHITECTURE 1 SIMPLE LSTM
nb_epoch = 5
batch_size = 32

model = simple_lstm()
model.fit(X_train, target_train, nb_epoch=nb_epoch, batch_size=batch_size)
loss, accuracy = model.evaluate(X_test, target_test, verbose=1)
print('\nFinal Cross-Validation Accuracy', accuracy, '\n')
print_layers_dims(model)

Epoch 1/5
7498/7498 [==============================] - 35s 5ms/step - loss: 0.6403 - acc: 0.7341
Epoch 2/5
7498/7498 [==============================] - 33s 4ms/step - loss: 0.5265 - acc: 0.7559
Epoch 3/5
7498/7498 [==============================] - 34s 4ms/step - loss: 0.4855 - acc: 0.7630
Epoch 4/5
7498/7498 [==============================] - 33s 4ms/step - loss: 0.4534 - acc: 0.7847
Epoch 5/5
2500/2500 [==============================] - 3s 1ms/step

Final Cross-Validation Accuracy 0.8088 

Input Shape:  (None, 75) Output Shape:  (None, 75)
Input Shape:  (None, 75) Output Shape:  (None, 75, 32)
Input Shape:  (None, 75, 32) Output Shape:  (None, 32)
Input Shape:  (None, 32) Output Shape:  (None, 32)
Input Shape:  (None, 32) Output Shape:  (None, 1)


### With inputs from Professor used a fully connected CNN 

In [109]:
def conv_fully(max_len=75, emb_dim=32, max_vocab_len=100, W_reg=regularizers.l2(1e-4)):
    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')
    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len,
                W_regularizer=W_reg)(main_input) 
    emb = Dropout(0.25)(emb)

    
    def sum_1d(X):
        return K.sum(X, axis=1)
    
    def get_conv_layer(emb, kernel_size=5, filters=256):
        # Conv layer
        conv = Convolution1D(kernel_size=kernel_size, filters=filters, \
                     border_mode='same')(emb)
        conv = ELU()(conv)

        conv = Lambda(sum_1d, output_shape=(filters,))(conv)
        #conv = BatchNormalization(mode=0)(conv)
        conv = Dropout(0.5)(conv)
        return conv
        
    # Multiple Conv Layers
    
    # calling custom conv function from above
    conv1 = get_conv_layer(emb, kernel_size=2, filters=256)
    conv2 = get_conv_layer(emb, kernel_size=3, filters=256)
    conv3 = get_conv_layer(emb, kernel_size=4, filters=256)
    conv4 = get_conv_layer(emb, kernel_size=5, filters=256)

    # Fully Connected Layers
    merged = concatenate([conv1,conv2,conv3,conv4], axis=1)

    hidden1 = Dense(1024)(merged)
    hidden1 = ELU()(hidden1)
    hidden1 = BatchNormalization(mode=0)(hidden1)
    hidden1 = Dropout(0.5)(hidden1)

    hidden2 = Dense(1024)(hidden1)
    hidden2 = ELU()(hidden2)
    hidden2 = BatchNormalization(mode=0)(hidden2)
    hidden2 = Dropout(0.5)(hidden2)
    
    # Output layer (last fully connected layer)
    output = Dense(1, activation='sigmoid', name='output')(hidden2)

    # Compile model and define optimizer
    model = Model(input=[main_input], output=[output])
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model

### Model Characteristics
1. 4 Fully connected layers. 
2. Activation function used is sigmoid. 
3. Optimizer in used is Adam
4. Loss function in use Binary Cross entropy. 
5. With  5 Epocs and a batch size of 32  is Final Cross-Validation Accuracy 0.81349206633 

In [110]:
# Fit model and Cross-Validation, ARCHITECTURE 3 CONV + FULLY CONNECTED
epochs = 5
batch_size = 32

model = conv_fully()
model.fit(X_train, target_train, epochs=epochs, batch_size=batch_size)
loss, accuracy = model.evaluate(X_test, target_test, verbose=1)

print('\nFinal Cross-Validation Accuracy', accuracy, '\n')
print_layers_dims(model)

Epoch 1/5
7498/7498 [==============================] - 19s 3ms/step - loss: 0.8402 - acc: 0.5910
Epoch 2/5
7498/7498 [==============================] - 18s 2ms/step - loss: 0.6775 - acc: 0.6771
Epoch 3/5
7498/7498 [==============================] - 18s 2ms/step - loss: 0.6026 - acc: 0.7238
Epoch 4/5
7498/7498 [==============================] - 20s 3ms/step - loss: 0.5804 - acc: 0.7457
Epoch 5/5
2500/2500 [==============================] - 2s 921us/step

Final Cross-Validation Accuracy 0.8124 

Input Shape:  (None, 75) Output Shape:  (None, 75)
Input Shape:  (None, 75) Output Shape:  (None, 75, 32)
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 32)
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 256)
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 256)
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 256)
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 256)
Input Shape:  (None, 75, 256) Output Shape:  (None, 75, 256)
Input Shape:  (None, 75, 256) Outpu

In [67]:
# GENERAL save model to disk function!
def save_model(fileModelJSON,fileWeights):
    #print("Saving model to disk: ",fileModelJSON,"and",fileWeights)
    #have h5py installed
    if Path(fileModelJSON).is_file():
        os.remove(fileModelJSON)
    json_string = model.to_json()
    with open(fileModelJSON,'w' ) as f:
        json.dump(json_string, f)
    if Path(fileWeights).is_file():
        os.remove(fileWeights)
    model.save_weights(fileWeights)
    

In [ ]:
model_name = "deeplearning_1DConv"
save_model(path + model_name + ".json", path + model_name + ".h5")
#model = load_model(path + model_name + ".json", path + model_name + ".h5")
print_layers_dims(model)